In [1]:
%reset
%env HSA_OVERRIDE_GFX_VERSION=10.3.0

env: HSA_OVERRIDE_GFX_VERSION=10.3.0


In [2]:
import os
import time

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchinfo import summary
from Classes import ConvLSTM

import matplotlib.pyplot as plt

# Parameters
TRAIN_DATASETS = 'Test/'
TEST_DATASET_FILE = 'Stock100_10.11.pt'
NUM_EPOCHS = 50

MODEL_NAME = 'work_please'
LATEST_MODEL_VERSION = 'BASE.pt'
CURRENT_MODEL_EPOCHS = 0

In [3]:
test_dataset = torch.load('Data/Datasets/Stock_100/MASTER.pt')

In [ ]:
conv_layer = nn.Conv1d(15, 32, kernel_size=3, device='cuda')
x = torch.randn(32, 15, 90).to('cuda')
print(x.size())
print(x.type())
y = conv_layer(x)
print(y.size())
print(y.type())

torch.Size([32, 15, 90])
torch.cuda.FloatTensor


In [3]:
model_params = {}

with open(f'Models/{MODEL_NAME}/params.txt', 'r') as f:
    for line in f:
        key, value = line.split(": ")
        if key in ['input_length', 'input_features', 'output_length', 'output_features',
                   'kernel_size', 'batch_size']:
            model_params[key] = int(value)
        elif key in ['conv_hidden_sizes', 'lstm_hidden_sizes']:
            # Removing brackets and spaces, then converting to int
            model_params[key] = list(map(int, value.replace('[','').replace(']','').replace(' ','').split(',')))
        else: # it's a string (model_name)
            model_params[key] = value.strip()  # .strip() to remove newline characters

In [4]:
print('Model parameters as follows:')
print(f'Input shape - ({model_params["input_length"]},{model_params["input_features"]})')
print(f'Output shape - ({model_params["output_length"]},{model_params["output_features"]})')
print('----------------------------------')
print(f'Conv hidden sizes - {model_params["conv_hidden_sizes"]}')
print(f'LSTM hidden sizes - {model_params["lstm_hidden_sizes"]}')
print(f'Kernel size - {model_params["kernel_size"]}')
print(f'Batch size - {model_params["batch_size"]}')

Model parameters as follows:
Input shape - (90,15)
Output shape - (1,1)
----------------------------------
Conv hidden sizes - [32, 64]
LSTM hidden sizes - [128, 128]
Kernel size - 3
Batch size - 32


In [5]:
print(torch.cuda.is_available())
device = torch.device('cuda')

True


In [6]:
model = ConvLSTM(input_features=model_params['input_features'],
                 conv_hidden_sizes=model_params['conv_hidden_sizes'],
                 lstm_hidden_sizes=model_params['lstm_hidden_sizes'],
                 kernel_size=model_params['kernel_size'],
                 output_features=model_params['output_features']
                 )
model.load_state_dict(torch.load(f'Models/{MODEL_NAME}/{LATEST_MODEL_VERSION}'))
model.to(device)
summary(model)

Layer (type:depth-idx)                   Param #
ConvLSTM                                 --
├─ModuleList: 1-1                        --
│    └─Conv1d: 2-1                       1,472
│    └─Conv1d: 2-2                       6,208
├─ModuleList: 1-2                        --
│    └─LSTM: 2-3                         99,328
│    └─LSTM: 2-4                         132,096
├─Conv1d: 1-3                            1,472
├─Linear: 1-4                            129
├─Dropout: 1-5                           --
Total params: 240,705
Trainable params: 240,705
Non-trainable params: 0

In [7]:
datasets = []

for concat_dataset in os.listdir(f'Data/Datasets/{TRAIN_DATASETS}'):
    dataset = torch.load(f'Data/Datasets/{TRAIN_DATASETS}/{concat_dataset}')
    datasets.extend(dataset.datasets)
    print(f'Loaded {concat_dataset} !!!')

Loaded Stock100_10.11.pt !!!


In [8]:
train_dataset = torch.utils.data.ConcatDataset(datasets)
test_dataset = torch.load(f'Data/Datasets/Test/{TEST_DATASET_FILE}')

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=model_params["batch_size"],shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=model_params["batch_size"],shuffle=False, drop_last=True)

In [10]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
test_losses = []
epoch_times = []

In [ ]:
for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    train_loss = 0

    # Train for epoch
    model.train()
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.float().to(device), targets.float().to(device)

        optimizer.zero_grad()
        outputs = model(inputs).float()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_dataloader)
    train_losses.append(train_loss)

    # Evaluate for epoch
    with torch.no_grad():
        test_loss = 0
        model.eval()

        for inputs, targets in test_dataloader:
            print('Testing!!!')
            inputs, targets = inputs.float().to(device), targets.float().to(device)

            outputs = model(inputs).float()
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            print(f'Loss: {loss.item()}')

        test_loss /= len(test_dataloader)
        test_losses.append(test_loss)

    epoch_time = time.time() - start_time
    epoch_times.append(epoch_time)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f} | Time taken: {epoch_time:.1f} seconds")

    torch.save(model.state_dict(), f'Models/{MODEL_NAME}/{MODEL_NAME}_epoch_{CURRENT_MODEL_EPOCHS+epoch+1}.pt')

----- Training!!! -----
Inputs shape: torch.Size([32, 90, 15])
Trying defined conv
SUCCESS!!! Temp shape: torch.Size([32, 32, 88])
Zero grad!!!
Before transpotition:  torch.Size([32, 90, 15])
After transpotition:  torch.Size([32, 15, 90])
torch.Size([32, 15, 90])
torch.cuda.FloatTensor


In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)